In [1]:
using Compat, Random, Distributions, Plots, LinearAlgebra, Statistics, LaTeXStrings, SparseArrays, CPUTime, JLD2
using SpecialFunctions, GSL, SuiteSparse, Distributed, Roots, NBInclude
gr(fmt=:png)
e = 2.7182818284
Random.seed!(1);

In [2]:
addprocs(2)
@everywhere using Compat, Random, Distributions, Plots, LinearAlgebra, Statistics, LaTeXStrings, SparseArrays, CPUTime, JLD2, SpecialFunctions, GSL, SuiteSparse, Distributed
@everywhere using SharedArrays, ParallelDataTransfer, Roots, NBInclude
@everywhere @nbinclude("setup.ipynb")
@everywhere using .setup, .functions
procs()

3-element Vector{Int64}:
 1
 2
 3

In [3]:
# set the time grid
T = 600
T_bar = 20
dt = T/1200
time_grid = collect(0:dt:T)
L = length(time_grid)
t_grid = dt, L;

Options for productivity parameters (important to load the relevant steady state):

* baseline productivity differences: productivity in non-tradables is 0.6 of that in tradables
* equal productivities
* productivities switched: productivity in tradables is 0.6 of that in non-tradables

Choose productivity_gap from mutually exclusive "**baseline**", "**equal**", "**switched**" (uncomment the relevant one):

In [4]:
productivity_gap = "baseline"
#productivity_gap = "equal"
#productivity_gap = "switched"

# loading parameters
if productivity_gap == "baseline"
    @load "external_parameters.jld2"
    @load "steady_state_output.jld2"
elseif productivity_gap == "equal"
    @load "external_parameters_equal.jld2"
    @load "steady_state_output_equal.jld2"
elseif productivity_gap == "switched"
    @load "external_parameters_switched.jld2"
    @load "steady_state_output_switched.jld2"
end

prod_shifters = prod_N, prod_T
ss_objects = q_N, q_F, q_H, q_E, q_T, q_total, G, consumption, debt, loan_arbitrage, distr_T, value_T, cons_T, labor_supply_T, distr_N, value_N, cons_N, labor_supply_N;

Options for dynamics include

* setting $\{\theta,\theta_g,\theta_e\}$ (equal to $\{1.5,3.0,3.0\}$ by default)
* setting $\kappa$ (equal to $0.577$ by default)
* choosing exchange rate regime from mutually exclusive {*flexible*,*peg*,*ner_targeting*}
    - if *ner_targeting* is chosen, $ϕ_ϵ$ has to be set (0 corresponds to pure float)
* choosing fiscal regime from mutually exclusive {*taxes_adjust*,*gov_adjust*}
    - if *taxes_adjust* is chosen, the tax allocation mechanism must be chosen from {*flat*,*prop*}
    
Pre-computed sequences in the folder "transition_results":

| description | file name | monetary indicator | $\phi_\epsilon$ | $\kappa$ | $\{\theta,\theta_g,\theta_e\}$ | fiscal regime | tax distribution | productivities |
|---|---|---|---|---|---|---|---|---|
|baseline <font color='blue'>float</font>|"transition_results/float.jld2"|*ner_targeting*|0|0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|baseline <font color='red'>peg</font>|"transition_results/peg.jld2"|*peg*| |0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='blue'>float</font> with equal productivity in N and T|"transition_results/float_equal.jld2"|*ner_targeting*|0|0.577|{1.5,3.0,3.0}|*gov_adjust*| | equal |
|<font color='red'>peg</font> with equal productivity in N and T|"transition_results/peg_equal.jld2"|*peg*| |0.577|{1.5,3.0,3.0}|*gov_adjust*| | equal |
|<font color='blue'>float</font> with productivity in N higher than in T|"transition_results/float_switched.jld2"|*ner_targeting*|0|0.577|{1.5,3.0,3.0}|*gov_adjust*| | switched |
|<font color='red'>peg</font> with productivity in N higher than in T|"transition_results/peg_switched.jld2"|*peg*| |0.577|{1.5,3.0,3.0}|*gov_adjust*| | switched |
|<font color='blue'>float</font> with proportional tax adjustment|"transition_results/float_prop.jld2"|*ner_targeting*|0|0.577|{1.5,3.0,3.0}|*tax_adjust*| *prop* | baseline |
|<font color='red'>peg</font> with proportional tax adjustment|"transition_results/peg_prop.jld2"|*peg*| |0.577|{1.5,3.0,3.0}|*tax_adjust*| *prop* | baseline |
|<font color='blue'>float</font> with flat tax adjustment|"transition_results/float_flat.jld2"|*ner_targeting*|0|0.577|{1.5,3.0,3.0}|*tax_adjust*| *flat* | baseline |
|<font color='red'>peg</font> with flat tax adjustment|"transition_results/peg_flat.jld2"|*peg*| |0.577|{1.5,3.0,3.0}|*tax_adjust*| *flat* | baseline |
|<font color='blue'>float</font> with a higher slope of the Phillips curve|"transition_results/float_high_phil.jld2"|*ner_targeting*|0|0.577$\cdot$1.25|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='red'>peg</font> with a higher slope of the Phillips curve|"transition_results/peg_high_phil.jld2"|*peg*| |0.577$\cdot$1.25|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='blue'>float</font> with a lower slope of the Phillips curve|"transition_results/float_low_phil.jld2"|*ner_targeting*|0|0.577$\cdot$0.75|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='red'>peg</font> with a lower slope of the Phillips curve|"transition_results/peg_low_phil.jld2"|*peg*| |0.577$\cdot$0.75|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.75$|"transition_results/mix_075.jld2"|*ner_targeting*|0.75|0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.50$|"transition_results/mix_050.jld2"|*ner_targeting*|0.5|0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.25$|"transition_results/mix_025.jld2"|*ner_targeting*|0.25|0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.25$|"transition_results/mix_025_neg.jld2"|*ner_targeting*|-0.25|0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.50$|"transition_results/mix_050_neg.jld2"|*ner_targeting*|-0.5|0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.75$|"transition_results/mix_075_neg.jld2"|*ner_targeting*|-0.75|0.577|{1.5,3.0,3.0}|*gov_adjust*| | baseline |
|<font color='blue'>float</font> with high elasticities|"transition_results/float_high_elast.jld2"|*ner_targeting*|0|0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='red'>peg</font> with high elasticities|"transition_results/peg_high_elast.jld2"|*peg*| |0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.75$ and high elasticities|"transition_results/high_075.jld2"|*ner_targeting*|0.75|0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.50$ and high elasticities|"transition_results/high_050.jld2"|*ner_targeting*|0.5|0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.25$ and high elasticities|"transition_results/high_025.jld2"|*ner_targeting*|0.25|0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.25$ and high elasticities|"transition_results/high_025_neg.jld2"|*ner_targeting*|-0.25|0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.50$ and high elasticities|"transition_results/high_050_neg.jld2"|*ner_targeting*|-0.5|0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.75$ and high elasticities|"transition_results/high_075_neg.jld2"|*ner_targeting*|-0.75|0.577|{1.5,3.0,3.0}$\cdot$2|*gov_adjust*| | baseline |
|<font color='blue'>float</font> with low elasticities|"transition_results/float_low_elast.jld2"|*ner_targeting*|0|0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |
|<font color='red'>peg</font> with low elasticities|"transition_results/peg_low_elast.jld2"|*peg*| |0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.75$ and low elasticities|"transition_results/low_075.jld2"|*ner_targeting*|0.75|0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.50$ and low elasticities|"transition_results/low_050.jld2"|*ner_targeting*|0.5|0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=0.25$ and low elasticities|"transition_results/low_025.jld2"|*ner_targeting*|0.25|0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.25$ and low elasticities|"transition_results/low_025_neg.jld2"|*ner_targeting*|-0.25|0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.50$ and low elasticities|"transition_results/low_050_neg.jld2"|*ner_targeting*|-0.5|0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |
|<font color='purple'>mixed regime</font> with $\phi_\epsilon=-0.75$ and low elasticities|"transition_results/low_075_neg.jld2"|*ner_targeting*|-0.75|0.577|{1.5,3.0,3.0}$\cdot$0.5|*gov_adjust*| | baseline |



In [93]:
par.θ = 1.5
par.θ_g = 3.0
par.θ_e = 3.0
par.κ = 0.0067/par.ρ
flexible, peg, ner_targeting = false, false, true
ϕ_ϵ = 0.0
taxes_adjust, gov_adjust = false, true
flat, prop = false, true

indicators = flexible, peg, ner_targeting, taxes_adjust, gov_adjust, flat, prop
ψ = 0.02/12*exp.(-time_grid/12*1.15)

# initialize sequences
wT_sequence, wN_sequence, marg_sequence = ones(L), ones(L), ones(L)
excise_sequence, gap_N, gap_T, gap_m, gap_μ = zeros(L), zeros(L), zeros(L), zeros(L), zeros(L)
rd_sequence = par.r_d*ones(L) + ψ/2
initial_sequences = wT_sequence, wN_sequence, marg_sequence, rd_sequence, excise_sequence, gap_N, gap_T, gap_m, gap_μ

# alternatively, load pre-computed sequences (comment the following lines to run the transition routine from flat guesses)
@load "transition_results/float.jld2"
initial_sequences = res_this.wT_sequence, res_this.wN_sequence, res_this.marg_sequence, res_this.rd_sequence, res_this.excise_sequence, res_this.gap_N, res_this.gap_T, res_this.gap_m, zeros(L);

In [96]:
# run the routine
max_iter = 1200
toler = 1e-6
max_T, max_N, max_m, mean_T, mean_N, mean_m, res_this = transition(initial_sequences,ψ,indicators,ϕ_ϵ,par,par_grid,prod_shifters,ss_objects,t_grid,max_iter,toler);

iteration 1
 21.445221 seconds (1.17 k allocations: 550.605 MiB, 0.16% gc time)
workers iterated;
tradables: mean gap = 1.2584214305674286e-7, max gap = 9.543849033288865e-7, gap at 0 = 9.543849033288865e-7
non-tradables: mean gap = 1.177431250935692e-7, max gap = 8.706700167504522e-7, gap at 0 = -8.706700167504522e-7
marginal cost: mean gap = 3.907093754829421e-10, max gap = 5.468077324266574e-9, gap at 0 = -5.468077324266574e-9


In [95]:
# save results
@save "transition_results/float.jld2" res_this